In [6]:
using DrWatson
@quickactivate "sst-mot-analysis"

import DrWatson: datadir, srcdir
import DrWatson: @quickactivate

using GH19, TMI, PythonCall, Revise
using CSV, DataFrames, NaNStatistics, NCDatasets, LinearAlgebra, SparseArrays

In [7]:
function config_from_nc(TMIfile; compute_lu = true)

    println("Form water-mass matrix A")
    @time A = watermassmatrix(TMIfile)

    # LU factorization for efficient matrix inversions
    if compute_lu
        println("LU factorization of A")
        @time Alu = lu(A)
    else
        println("skip LU factorization of A")
        Alu = nothing
    end

    γ = Grid(TMIfile)
    
    # would be good to make this optional
    println("circulation matrix L=")
    @time L = circulationmatrix(TMIfile,A,γ)

    println("Boundary matrix B=")
    @time B = boundarymatrix(TMIfile,γ)
    
    return  A, Alu, γ, TMIfile, L, B
end

config_from_nc (generic function with 1 method)

In [8]:
function get_MOT_and_SST(TMIfile::String, γ::Grid)
    θ = readfield(TMIfile, "θ", γ); #true mean
    volume = cellvolume(γ).tracer 
    area = cellarea(γ).tracer 

    θ̄_MOT = nansum(θ.tracer .* volume) / nansum(volume)
    θ_SST = getsurfaceboundary(θ);
    θ̄_SST = nansum(θ_SST.tracer .* area) / nansum(area)
    return θ̄_MOT, θ̄_SST
end

function get_MOT_and_SST(θ::Field, γ::Grid)
    volume = cellvolume(γ).tracer 
    area = cellarea(γ).tracer 

    θ̄_MOT = nansum(θ.tracer .* volume) / nansum(volume)
    θ_SST = getsurfaceboundary(θ);
    θ̄_SST = nansum(θ_SST.tracer .* area) / nansum(area)
    return θ̄_MOT, θ̄_SST
end

get_MOT_and_SST (generic function with 2 methods)

In [49]:
###### LOAD LGM NO DROP###########
TMI_file = "TMI_LGM_90x45x33_G14_nosealeveldrop.nc"

A, Alu, γ, TMIfile, L, B = config_from_nc(datadir(TMI_file))

θ_nodrop = readfield(datadir(TMI_file), "θ", γ)

Form water-mass matrix A
  0.033192 seconds (4.36 k allocations: 34.891 MiB, 22.19% gc time)
LU factorization of A
  0.613776 seconds (88 allocations: 492.957 MiB, 0.07% gc time)
circulation matrix L=
  0.002678 seconds (764 allocations: 63.273 KiB)
Boundary matrix B=
  0.001867 seconds (763 allocations: 63.250 KiB)


Field{Float64, Float64, 3, Array{Float64, 3}}
Field size (90, 45, 33)
Surface view
                     potential temperature                      
      ┌────────────────────────────────────────────────┐  30    
   45 │▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄│ ┌──┐   
      │▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄│ │▄▄│   
      │▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄│ │▄▄│   
      │▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄│ │▄▄│   
      │▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄│ │▄▄│   
      │▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄│ │▄▄│   
      │▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄│ │▄▄│ °C
      │▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄│ │▄▄│   
      │▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄│ │▄▄│   
      │▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄│ │▄▄│   
      │▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄│ │▄▄│   
    1 │▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄│ └──┘   
      └

In [50]:
###### LOAD LGM ###########
TMI_version = "LGM_90x45x33_G14";
A, Alu_dropped, γ_lgm, TMIfile, L, B = config(TMI_version); println(sum(cellvolume(γ_lgm))) 
TMIfile = TMI.pkgdatadir("TMI_"*TMI_version*".nc")

θ_dropped = readfield(datadir(TMI_file), "θ", γ_lgm)

θ̄_LGM_MOT, θ̄_LGM_SST = get_MOT_and_SST(TMIfile, γ_lgm);

Form water-mass matrix A
  0.019372 seconds (1.99 k allocations: 26.437 MiB, 31.97% gc time)
LU factorization of A
  0.342375 seconds (88 allocations: 305.217 MiB, 0.09% gc time)
circulation matrix L=
  0.001614 seconds (333 allocations: 26.828 KiB)
Boundary matrix B=
  0.001334 seconds (332 allocations: 26.805 KiB)
1.4033505303988716e18
readfield warning: non-NaN value off grid
resetting to NaN


In [53]:
SST_nodrop = 1. * getsurfaceboundary(θ_nodrop)
SST_nodrop.tracer[.!γ_lgm.wet[:, :, surfaceindex(γ_lgm)]] .= NaN;

SST_dropped = 1. * getsurfaceboundary(θ_dropped)
SST_dropped.tracer .= 1.0 .* (SST_nodrop.tracer);

In [56]:
θ_mod_bc = steadyinversion(Alu_dropped,SST_dropped,γ_lgm)


Field{Float64, Float64, 3, Array{Float64, 3}}
Field size (90, 45, 33)
Surface view
                      boundary condition                             
      ┌────────────────────────────────────────────────┐  30         
   45 │▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄│ ┌──┐        
      │▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄│ │▄▄│        
      │▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄│ │▄▄│        
      │▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄│ │▄▄│        
      │▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄│ │▄▄│        
      │▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄│ │▄▄│        
      │▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄│ │▄▄│ unknown
      │▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄│ │▄▄│        
      │▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄│ │▄▄│        
      │▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄│ │▄▄│        
      │▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄│ │▄▄│        
    1 │

In [59]:
TMI.mean(θ_nodrop)

1.2715820754691407

In [60]:
TMI.mean(θ_mod_bc)

1.338476573058139

In [61]:
θ̄_LGM_MOT

0.929498148418711